<center>
<h1><b>Lab 2</b></h1>
<h1>PHYS 580 - Computational Physics</h1>
<h2>Professor Molnar</h2>
</br>
<h3><b>Ethan Knox</b></h3>
<h4>https://www.github.com/ethank5149</h4>
<h4>ethank5149@gmail.com</h4>
</br>
</br>
<h3><b>September 11, 2020</b></h3>
</center>

### Imports

In [1]:
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from functools import partial

### Euler Step

In [2]:
def euler_step(f, y, t, dt):
    return y + f(t, y) * dt

### Parameters

In [3]:
g = 9.81
v0 = 700
dt = 0.01
dtheta = 0.01
b2_m = 4.0e-5
y0scale = 1.0e4
alpha = 6.5e-3
gamma = 1.4
T_grd = 293

theta0 = 5
theta1 = 85
ntheta = round((theta1 - theta0) / dtheta)
# termination_condition = lambda _t, _y : _t > 0. and _y[1] <= 0.
terminate = lambda _t, _y : _t * _y[1] < 0
terminate.terminal = True
terminate.direction = -1
step = euler_step

### DSolve Function

In [4]:
def dsolve(f, t, y0):
    t = np.asarray(t)  # Ensure t is a Numpy array
    y = np.zeros((np.size(t), np.size(y0)))  # Create our output data container
    y[0] = y0  # Set initial condition
    terminated_at = -1  # Index of the terminated point

    for i in range(np.size(t)-1):
        y[i+1] = step(f, y[i], t[i], t[i+1] - t[i])  # Step forward

        if terminate(t[i], y[i]):  # Check termination condition
            terminated_at = i  # Set termination point
            break
    
    return t[:terminated_at], y[:terminated_at, :]

### Difference Functions

In [5]:
def center_difference(f, x_i):
    return (f(x_i + 0.5 * dtheta) - f(x_i - 0.5 * dtheta)) / dtheta

def forward_difference(f, x_i):
    return (f(x_i + dtheta) - f(x_i)) / dtheta

def backward_difference(f, x_i):
    return (f(x_i) - f(x_i - dtheta)) / dtheta

### Bisection Algorithm

In [6]:
def bisection(f, a, b, N):
    
    if f(a) * f(b) >= 0:
        return None
    
    a_n, b_n = a, b
    
    for n in range(1, N + 1):
        m_n = (a_n + b_n) / 2
        f_m_n = f(m_n)
        
        if f(a_n) * f_m_n < 0:
            b_n = m_n
        elif f(b_n) * f_m_n < 0:
            a_n = m_n
        elif f_m_n == 0:
            return m_n
        else:
            return None
    
    return (a_n + b_n) / 2

## Drag Forces

In [7]:
def df_isothermal_drag(x,  y):
    v = np.sqrt(y[2]**2 + y[3]**2)
    drag_factor = -b2_m * v * np.exp(-y[1] / y0scale)
    return np.asarray([y[2], y[3], drag_factor * y[2], -g + drag_factor * y[3]])


def df_adiabatic_drag(x, y):
    v = np.sqrt(y[2]**2 + y[3]**2)
    drag_factor = -b2_m * v * (1 - alpha * y[1]/T_grd)**(1/ (gamma-1))
    return np.asarray([y[2], y[3], drag_factor * y[2], -g + drag_factor * y[3]])


def df_no_drag(x, y):
    return np.asarray([y[2], y[3], 0, -g])


def df_constant_drag(x, y):
    v = np.sqrt(y[2]**2 + y[3]**2)
    drag_factor = -b2_m * v
    return np.asarray([y[2], y[3], drag_factor * y[2], -g + drag_factor * y[3]])

## "Shooting" Function

In [8]:
def shoot(forcing, theta):
    # Convert initial conditions into an initial velocity
    vx0 = v0 * np.cos(theta / 180. * np.pi)  
    vy0 = v0 * np.sin(theta / 180. * np.pi)
    y0 = np.asarray([0., 0., vx0, vy0])  # Set initial condition

    theoretical_max_range = v0**2 / g   # max range of shell, in vaccuum (for automatic horizontal plot range)
    theoretical_max_flight_time = theoretical_max_range / y0[2]  # flight time at maxr, in vacuum (for atomatic calculation end time)

    nsteps = round(theoretical_max_flight_time / dt)   # time steps
    t = np.linspace(0, theoretical_max_flight_time, int(nsteps))
    
    t, y_soln = dsolve(forcing, t, y0)
    y_soln = y_soln.T

    x, y, dx, dy = y_soln[0], y_soln[1], y_soln[2], y_soln[3]

    max_range = (y[-1] * x[-2] - y[-2] * x[-1] ) / (y[-1] - y[-2])
    x[-1] = max_range
    y[-1] = 0.
    
    max_height_index = np.argmax(y)
    max_height = y[max_height_index]
    x_at_max_height = x[max_height_index]

    return x, y, dx, dy, x_at_max_height, max_height, max_range

In [9]:
def max_range(forcing, theta):
    soln = shoot(forcing, theta)
    return soln[6]

def d_max_range(forcing, dtheta, theta):
    return center_difference(partial(max_range, forcing), theta)

## Sweep Optimum Angle

In [10]:
# function_to_zero = partial(d_max_range, df_no_drag, dtheta)
# no_drag_optimum_angle = bisection(function_to_zero, theta0, theta1, ntheta)
# print(no_drag_optimum_angle)

# function_to_zero = partial(d_max_range, df_constant_drag, dtheta)
# constant_drag_optimum_angle = bisection(function_to_zero, theta0, theta1, ntheta)
# print(constant_drag_optimum_angle)

# function_to_zero = partial(d_max_range, df_adiabatic_drag, dtheta)
# adiabatic_drag_optimum_angle = bisection(function_to_zero, theta0, theta1, ntheta)
# print(adiabatic_drag_optimum_angle)

# function_to_zero = partial(d_max_range, df_isothermal_drag, dtheta)
# isothermal_drag_optimum_angle = bisection(function_to_zero, theta0, theta1, ntheta)
# print(isothermal_drag_optimum_angle)

In [11]:
# no_drag = shoot(df_no_drag, no_drag_optimum_angle)
# constant_drag = shoot(df_constant_drag, constant_drag_optimum_angle)
# adiabatic_drag = shoot(df_adiabatic_drag, adiabatic_drag_optimum_angle)
# isothermal_drag = shoot(df_isothermal_drag, isothermal_drag_optimum_angle)

## Plotting

### Sweeping Optimum Launch Angle

In [12]:
# fig = plt.figure(figsize=(16,9), constrained_layout=True)
# plt.xlabel('Angle [degrees]')
# plt.ylabel('Range [m]')
# plt.suptitle("Ranges")
# plt.plot(no_drag[0], no_drag[1], label = "No Drag")
# plt.plot(constant_drag[0], constant_drag[1], label = "Constant Drag")
# plt.plot(adiabatic_drag[0], adiabatic_drag[1], label = "Adiabatic Drag")
# plt.plot(isothermal_drag[0], isothermal_drag[1], label = "Isothermal Drag")
# plt.legend()
# plt.grid()
# plt.show()

## Faster Method

In [13]:
from scipy.integrate import solve_ivp
from scipy.misc import derivative
from scipy.optimize import root_scalar

In [21]:
# terminate = lambda _t, _y : _t * _y[1] < 0

myforce = df_adiabatic_drag

def get_range(theta, force):
    vx0 = v0 * np.cos(theta / 180. * np.pi)  
    vy0 = v0 * np.sin(theta / 180. * np.pi)
    y0 = np.asarray([0., 0., vx0, vy0])
    
    theoretical_max_range = v0**2 / g
    theoretical_max_flight_time = theoretical_max_range / y0[2]
    
    soln = solve_ivp(force, (0.0, theoretical_max_flight_time), y0, events=terminate)
    return soln.y[-1][0]
    
#     soln = solve_ivp(force, (0, theoretical_max_flight_time), y0)
#     return np.max(np.asarray([y[0] for t, y in zip(soln.t, soln.y.T) if terminate(t, y)]))


def get_drange(theta, force):
    return derivative(get_range, x0=theta, dx=dtheta, args=(force,))


# angles = np.linspace(theta0, theta1, ntheta)
# f = partial(get_range, force=myforce)
# plt.plot(angles, np.asarray([f(angle) for angle in angles]))
# plt.show()

optimum_angle = root_scalar(get_drange, args=(myforce,), bracket=[theta0, theta1], x0=theta0, x1=theta1, method='secant')
print(optimum_angle)

      converged: True
           flag: 'converged'
 function_calls: 7
     iterations: 6
           root: 90.00000000000425


In [15]:
angles = np.linspace(theta0, theta1, ntheta)
# defect = [partial(get_drange, force=myforce)(angle) for angle in angles]
# print(defect)

In [16]:
angle = 45
def shoot(theta, force):
    vx0 = v0 * np.cos(theta / 180. * np.pi)  
    vy0 = v0 * np.sin(theta / 180. * np.pi)
    y0 = np.asarray([0., 0., vx0, vy0])
    
    theoretical_max_range = v0**2 / g
    theoretical_max_flight_time = theoretical_max_range / y0[2]
    
    return solve_ivp(force, (0, theoretical_max_flight_time), y0, max_step=dtheta, events=terminate)
sol = shoot(angle, myforce)
assert(terminate(sol.t[-1],sol.y.T[-1]))
print(np.shape(sol.y))
plt.plot(sol.y[0], sol.y[1])
plt.show()

AssertionError: 